In [1]:
import pathlib
from pathlib import Path
import pandas as pd
import xarray as xr
import numpy as np
from peak_resolver.process_chromeleon import process_chromeleon_file

In [2]:
def get_fanny_concentration(file: pathlib.PosixPath):
    n = file.name
    conc = {'1':0.05, '2':0.1, '3': 0.2, '4':0.3, '5': 0.4, '6':0.5}
    return conc[n[1]]

def read_fanny_calib(p: pathlib.PosixPath, tmin=0, tmax=30, tgrid=None):
    files = p.glob('*.txt')
    acid = p.name.split(' ')[0].lower().capitalize()
    print(acid)
    data_list = []
    for file in files:
        data = process_chromeleon_file(file, tmin, tmax, tgrid = tgrid)
        data[acid] = get_fanny_concentration(file)
        data = data.set_index(['t', acid]).to_xarray()

        
        
        data_list.append(data)
    data = xr.combine_by_coords(data_list)
    savepath = str(p / acid) + '.nc'
    data.to_netcdf(savepath)
    return files, data, acid

In [3]:
path = Path('/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/Calibration Curves/')
path = path.resolve()
acids = [i for i in path.iterdir() if i.is_dir()]
acids = tuple(acids)

files = []
data = []
names = []
# [print(i.name) for i in acids]
for acid in acids:
    # print(acid.name)

    f, d, name = read_fanny_calib(acid, tmin=6.5, tmax=9, 
                            tgrid = np.linspace(0.15, 19.85, 5997)
                           )
    files.append(f)
    data.append(d)
    names.append(name)
    

Acetic
              t
0      0.150000
1      0.153286
2      0.156571
3      0.159857
4      0.163142
...         ...
5992  19.836858
5993  19.840143
5994  19.843429
5995  19.846714
5996  19.850000

[5997 rows x 1 columns]
              t         s
0      0.150000 -0.000030
1      0.153286 -0.000038
2      0.156571 -0.000040
3      0.159857 -0.000032
4      0.163142 -0.000030
...         ...       ...
5992  19.836858  0.000611
5993  19.840143  0.000285
5994  19.843429 -0.000040
5995  19.846714 -0.000365
5996  19.850000 -0.000682

[5997 rows x 2 columns]
              t
0      0.150000
1      0.153286
2      0.156571
3      0.159857
4      0.163142
...         ...
5992  19.836858
5993  19.840143
5994  19.843429
5995  19.846714
5996  19.850000

[5997 rows x 1 columns]
              t         s
0      0.150000 -0.001058
1      0.153286 -0.001078
2      0.156571 -0.001097
3      0.159857 -0.001117
4      0.163142 -0.001128
...         ...       ...
5992  19.836858 -0.011379
5993  19.84014

In [4]:
tmin = 6.5
tmax = 9
a_path, ace = read_fanny_calib(acids[0], tmin=tmin, tmax=tmax)
b_path, but = read_fanny_calib(acids[1], tmin=tmin, tmax=tmax)
p_path, pro = read_fanny_calib(acids[2], tmin=tmin, tmax=tmax)
v_path, val = read_fanny_calib(acids[3], tmin=tmin, tmax=tmax)
c_path, cap = read_fanny_calib(acids[4], tmin=tmin, tmax=tmax)

Acetic
             t
0     6.500000
1     6.500278
2     6.500556
3     6.500834
4     6.501111
...        ...
8994  8.998889
8995  8.999166
8996  8.999444
8997  8.999722
8998  9.000000

[8999 rows x 1 columns]
             t         s
0     6.500000 -0.046588
1     6.500278 -0.046567
2     6.500556 -0.046546
3     6.500834 -0.046526
4     6.501111 -0.046506
...        ...       ...
8994  8.998889  0.018132
8995  8.999166  0.018118
8996  8.999444  0.018104
8997  8.999722  0.018090
8998  9.000000  0.018076

[8999 rows x 2 columns]
             t
0     6.500000
1     6.500278
2     6.500556
3     6.500834
4     6.501111
...        ...
8994  8.998889
8995  8.999166
8996  8.999444
8997  8.999722
8998  9.000000

[8999 rows x 1 columns]
             t         s
0     6.500000 -0.058218
1     6.500278 -0.058203
2     6.500556 -0.058188
3     6.500834 -0.058173
4     6.501111 -0.058158
...        ...       ...
8994  8.998889 -0.003817
8995  8.999166 -0.003820
8996  8.999444 -0.003822
8997  8.

ValueError: too many values to unpack (expected 2)

In [ ]:
for n, ds in zip(names, data): # save them to file
    q = n.upper() + ' ACID/' + n.lower() + '.nc'
    q = path / q
    ds.to_netcdf(q)